<a href="https://colab.research.google.com/github/shraddha-an/nlp/blob/main/so_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **BERT for Text Classification**

Implementing BERT model via Simple Transformers library to classify Stack Overflow Questions

Dataset: [Stack Overflow Questions](https://www.kaggle.com/imoore/60k-stack-overflow-questions-with-quality-rate)

Other models in this series:


1.   [Training an Embedding](https://github.com/shraddha-an/nlp/blob/main/word_embedding_classification.ipynb)
2.   [Pre-trained GloVe Embedding](https://github.com/shraddha-an/nlp/blob/main/pretrained_glove_classification.ipynb)
3. [fastText Classifier](https://github.com/shraddha-an/nlp/blob/main/so_fasttext.ipynb)




In [ ]:
# Installing simple transformers library
!pip install simpletransformers

# Data Manipulation/ Handling
import pandas as pd, numpy as np

# NLP Preprocessing
from gensim.utils import simple_preprocess

# BERT/Transformer
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [11]:
# Importing the dataset
dataset = pd.read_csv('train.csv')[['Body', 'Y']].rename(columns = {'Body': 'questions', 'Y': 'category'})
ds = pd.read_csv('valid.csv')[['Body', 'Y']].rename(columns = {'Body': 'questions', 'Y': 'category'})



dataset.tail(5), ds.head(5)

(                                               questions  category
 44995  <p>I am new to this and I am asking for help t...  LQ_CLOSE
 44996  <p>I am working on learning Python and was won...  LQ_CLOSE
 44997  <p>It looks like it costs 8 days per month in ...  LQ_CLOSE
 44998  <p>"I _____ any questions."</p>\n\n<p>I want t...  LQ_CLOSE
 44999  <p>I'm very new to programming and I'm teachin...  LQ_CLOSE,
                                            questions category
 0  I am having 4 different tables like \r\nselect...  LQ_EDIT
 1  I have two table m_master and tbl_appointment\...  LQ_EDIT
 2  <p>I'm trying to extract US states from wiki U...       HQ
 3  I'm so new to C#, I wanna make an application ...  LQ_EDIT
 4  basically i have this array:\r\n\r\n    array(...  LQ_EDIT)

In [12]:
# NLP preprocessing
from gensim.utils import simple_preprocess

dataset.iloc[:, 0] = dataset.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
ds.iloc[:, 0] = ds.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

In [13]:
# Label encoding the category column
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()

dataset.iloc[:, 1] = enc.fit_transform(dataset.iloc[:, 1])
ds.iloc[:, 1] = enc.transform(ds.iloc[:, 1])

# Printing again
ds.tail(4), dataset.head(4)

(                                               questions  category
 14996  try to multiply an integer by double but obtai...         1
 14997  urls py urls py file from django contrib impor...         2
 14998  have controller inside which server is connect...         1
 14999  so was recently helping someone out with some ...         1,
                                            questions  category
 0  already familiar with repeating tasks every se...         1
 1  like to understand why java optionals were des...         0
 2  am attempting to overlay title over an image w...         0
 3  the question is very simple but just could not...         0)

In [16]:
# Setting parameters for model configuration
model_args = ClassificationArgs(num_train_epochs = 3, overwrite_output_dir = True)

# Training the BERT model for classification
model = ClassificationModel('bert', 'bert-base-cased', num_labels = 3, args = model_args,
                            use_cuda = True)

# Training the model
model.train_model(dataset)



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(16875, 0.3611660744220829)

In [19]:
# Accuracy
from sklearn.metrics import accuracy_score

# Evaluate the model performance
results, model_output, wrong_predictions = model.eval_model(ds, acc = accuracy_score)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:851: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [20]:
print(results)

{'mcc': 0.7702724216712876, 'acc': 0.8467333333333333, 'eval_loss': 0.6148977538202501}
